# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [2]:
import logging
import os

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.data.dataset_factory import TabularDatasetFactory

from sklearn.model_selection import train_test_split

## Dataset

### Overview
This model will predict median house value from kaggle's data set "Calofornia housing"


### Workspace Set Up and Initialize an Experiment

In [4]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

quick-starts-ws-247912
aml-quickstarts-247912
southcentralus
48a74bb7-9950-4cc1-9caa-5d50f995cc55


In [5]:
os.listdir(os.curdir), os.getcwd()

(['.amlignore',
  '.amlignore.amltmp',
  '.ipynb_aml_checkpoints',
  'automl.ipynb',
  'automl.ipynb.amltmp',
  'automl_housing.pkl'],
 '/mnt/batch/tasks/shared/LS_root/mounts/clusters/notebook247912/code/Users/odl_user_247912/nd00333-capstone-master/nd00333-capstone-master/starter_file/automated_ml')

In [6]:
# choose a name for experiment
experiment_name = 'automl_housing'
project_folder = './automl-housing'
os.makedirs(project_folder, exist_ok=True)

experiment = Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
automl_housing,quick-starts-ws-247912,Link to Azure Machine Learning studio,Link to Documentation


### Create or Attach a Compute Resource

In [7]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "notebook247912"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)
# For a more detailed view of current AmlCompute status, use get_status().

Found existing cluster, use it.

Running


### Data set

In [15]:
import pandas as pd
data = Dataset.get_by_name(ws, name='housing')
data = data.to_pandas_dataframe()
data

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY
...,...,...,...,...,...,...,...,...,...,...
20635,-121.09,39.48,25.0,1665.0,374.0,845.0,330.0,1.5603,78100.0,INLAND
20636,-121.21,39.49,18.0,697.0,150.0,356.0,114.0,2.5568,77100.0,INLAND
20637,-121.22,39.43,17.0,2254.0,485.0,1007.0,433.0,1.7000,92300.0,INLAND
20638,-121.32,39.43,18.0,1860.0,409.0,741.0,349.0,1.8672,84700.0,INLAND


In [16]:
def prepare_data(data):    
    encoded_column=pd.get_dummies(data['ocean_proximity'], prefix='ocp')
    data=data.join(encoded_column)
    data=data.drop("ocean_proximity", axis=1)

    train, test = train_test_split(data, test_size=0.2, random_state=42)
    return train, test

train, test = prepare_data(data)
type(train), type(test), train.shape, test.shape

(pandas.core.frame.DataFrame,
 pandas.core.frame.DataFrame,
 (16512, 14),
 (4128, 14))

In [17]:
columns=train.columns
train.to_csv(path_or_buf='trainset.csv', columns=columns, header=True, index=False) 
test.to_csv(path_or_buf='testset.csv', columns=columns, header=True, index=False)

In [18]:
datastore=ws.get_default_datastore() 
datastore.upload_files(['trainset.csv'])

"datastore.upload_files" is deprecated after version 1.0.69. Please use "FileDatasetFactory.upload_directory" instead. See Dataset API change notice at https://aka.ms/dataset-deprecation.


Uploading an estimated of 1 files
Uploading trainset.csv
Uploaded trainset.csv, 1 files out of an estimated total of 1
Uploaded 1 files


$AZUREML_DATAREFERENCE_workspaceblobstore

In [19]:
train=TabularDatasetFactory.from_delimited_files([(datastore, 'trainset.csv')])

datastore.upload_files(['testset.csv'])

Uploading an estimated of 1 files
Uploading testset.csv
Uploaded testset.csv, 1 files out of an estimated total of 1
Uploaded 1 files


$AZUREML_DATAREFERENCE_workspaceblobstore

In [20]:
test=TabularDatasetFactory.from_delimited_files([(datastore, 'testset.csv')])

## AutoML Configuration

Since we are trying to predict a numeric value, in particular, "median_house_values", we have to define the task as "regression". Featurization is set to "auto" and we enable early stopping to avoid overfitting. We pass in our train and test sets and choose "normalized_mean_absolute_error" as the primary metric.

In [21]:
azureml.train.automl.utilities.get_primary_metrics("regression")

['r2_score',
 'spearman_correlation',
 'normalized_mean_absolute_error',
 'normalized_root_mean_squared_error']

In [22]:

automl_setting={
    "featurization": "auto",
    "experiment_timeout_minutes": 30,
    "enable_early_stopping": True,
    "verbosity": logging.INFO,
    "compute_target": compute_target
}

task="regression" 
automl_config = AutoMLConfig( 
    task=task, 
    primary_metric='normalized_root_mean_squared_error', 
    training_data=train, 
    validation_data = test, 
    label_column_name='median_house_value', 
    **automl_setting
)

## Submit Run & Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [23]:
# TODO: Submit your experiment
from azureml.widgets import RunDetails

automl_run = experiment.submit(automl_config, show_output=True)

RunDetails(automl_run).show()

Submitting remote run.
No run_configuration provided, running on notebook247912 with default configuration
Running on remote compute: notebook247912


Experiment,Id,Type,Status,Details Page,Docs Page
automl_housing,AutoML_dfc2bed6-b984-490e-ae53-7f0c1d8bf704,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

********************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.
              Learn more about high cardinality feature handling: https://aka.ms/AutomatedMLFeaturization

*************************************************

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [24]:
automl_run.wait_for_completion()

{'runId': 'AutoML_dfc2bed6-b984-490e-ae53-7f0c1d8bf704',
 'target': 'notebook247912',
 'status': 'Completed',
 'startTimeUtc': '2024-01-03T11:46:14.802175Z',
 'endTimeUtc': '2024-01-03T12:04:45.877873Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'No scores improved over last 10 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'normalized_root_mean_squared_error',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'notebook247912',
  'AMLSettingsJsonString': '{"path":null,"name":"automl_housing","subscription_id":"48a74bb7-9950-4cc1-9caa-5d50f995cc55","resource_group":"aml-quickstarts-247912","workspace_name":"quick-starts-ws-247912","region":"southcentralus",

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [25]:
# Retrieve and save your best automl model.
automl_run, fitted_automl_model = automl_run.get_output() 
print(fitted_automl_model) #To print
import joblib  
joblib.dump(fitted_automl_model, "automl_housing.pkl")

Package:azureml-automl-runtime, training version:1.52.0.post1, current version:1.51.0.post1
Package:azureml-core, training version:1.52.0, current version:1.51.0
Package:azureml-dataprep, training version:4.11.4, current version:4.10.8
Package:azureml-dataprep-rslex, training version:2.18.4, current version:2.17.12
Package:azureml-dataset-runtime, training version:1.52.0, current version:1.51.0
Package:azureml-defaults, training version:1.52.0, current version:1.51.0
Package:azureml-interpret, training version:1.52.0, current version:1.51.0
Package:azureml-mlflow, training version:1.52.0, current version:1.51.0
Package:azureml-pipeline-core, training version:1.52.0, current version:1.51.0
Package:azureml-responsibleai, training version:1.52.0, current version:1.51.0
Package:azureml-telemetry, training version:1.52.0, current version:1.51.0
Package:azureml-train-automl-client, training version:1.52.0, current version:1.51.0.post1
Package:azureml-train-automl-runtime, training version:1.

RegressionPipeline(pipeline=Pipeline(memory=None,
                                     steps=[('datatransformer',
                                             DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=False, is_onnx_compatible=False, observer=None, task='regression', working_dir='/mnt/batch/t...
                                             PreFittedSoftVotingRegressor(estimators=[('1', Pipeline(memory=None, steps=[('maxabsscaler', MaxAbsScaler(copy=True)), ('xgboostregressor', XGBoostRegressor(n_jobs=1, problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu'}), random_state=0, tree_method='auto'))], verbose=False)), ('0', Pipeline(memory=None, steps=[('maxabsscaler', MaxAbsScaler(copy=True)), ('lightgbmregressor', LightGBMRegressor(min_data_in_leaf=20, n_jobs=1, problem_info=ProblemInfo(gpu_training_param_dict={'p

['automl_housing.pkl']

In [26]:
os.listdir(os.curdir), os.getcwd()

(['.amlignore',
  '.amlignore.amltmp',
  '.azureml',
  '.ipynb_aml_checkpoints',
  'automl-housing',
  'automl.ipynb',
  'automl.ipynb.amltmp',
  'automl.log',
  'automl_housing.pkl',
  'azureml_automl.log',
  'testset.csv',
  'trainset.csv'],
 '/mnt/batch/tasks/shared/LS_root/mounts/clusters/notebook247912/code/Users/odl_user_247912/nd00333-capstone-master/nd00333-capstone-master/starter_file/automated_ml')

In [27]:
automl_run_metrics = automl_run.get_metrics()
automl_run_metrics

{'normalized_root_mean_squared_error': 0.09619700249370727,
 'root_mean_squared_log_error': 0.23057708337755914,
 'r2_score': 0.8338871313629255,
 'spearman_correlation': 0.9200313495801759,
 'root_mean_squared_error': 46655.738603453014,
 'explained_variance': 0.8339048748637179,
 'median_absolute_error': 20234.03860918009,
 'normalized_root_mean_squared_log_error': 0.06575588571652256,
 'normalized_mean_absolute_error': 0.06389772701377223,
 'mean_absolute_error': 30990.52539713356,
 'mean_absolute_percentage_error': 17.46132751931706,
 'normalized_median_absolute_error': 0.041719495196267414,
 'predicted_true': 'aml://artifactId/ExperimentRun/dcid.AutoML_dfc2bed6-b984-490e-ae53-7f0c1d8bf704_30/predicted_true',
 'residuals': 'aml://artifactId/ExperimentRun/dcid.AutoML_dfc2bed6-b984-490e-ae53-7f0c1d8bf704_30/residuals'}

In [28]:
print('Best Run ID', automl_run.id)
print('\n MAE', automl_run_metrics['mean_absolute_error'])

Bets Run ID AutoML_dfc2bed6-b984-490e-ae53-7f0c1d8bf704_30

 MAE 30990.52539713356


In [30]:
model = automl_run.register_model(model_name='automl-housing', model_path='Users/odl_user_247912/nd00333-capstone-master/nd00333-capstone-master/starter_file/automated_ml/automl_housing.pkl')

ModelPathNotFoundException: ModelPathNotFoundException:
	Message: Could not locate the provided model_path Users/odl_user_247912/nd00333-capstone-master/nd00333-capstone-master/starter_file/automated_ml/automl_housing.pkl in the set of files uploaded to the run: ['automl_driver.py', 'explanation/68cdc171/eval_data_viz.interpret.json', 'explanation/68cdc171/expected_values.interpret.json', 'explanation/68cdc171/features.interpret.json', 'explanation/68cdc171/global_names/0.interpret.json', 'explanation/68cdc171/global_rank/0.interpret.json', 'explanation/68cdc171/global_values/0.interpret.json', 'explanation/68cdc171/local_importance_values.interpret.json', 'explanation/68cdc171/rich_metadata.interpret.json', 'explanation/68cdc171/true_ys_viz.interpret.json', 'explanation/68cdc171/visualization_dict.interpret.json', 'explanation/68cdc171/ys_pred_viz.interpret.json', 'explanation/6d4ef608/eval_data_viz.interpret.json', 'explanation/6d4ef608/expected_values.interpret.json', 'explanation/6